# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.235166610389                   -0.50    8.0
  2   -7.250393036894       -1.82       -1.40    1.0
  3   -7.251274500886       -3.05       -2.08    4.0
  4   -7.251129355910   +   -3.84       -2.00    3.0
  5   -7.251333263869       -3.69       -2.77    2.0
  6   -7.251338189469       -5.31       -3.17    2.0
  7   -7.251338777787       -6.23       -3.98    1.0
  8   -7.251338793604       -7.80       -4.21    4.0
  9   -7.251338798449       -8.31       -4.87    2.0
 10   -7.251338798686       -9.62       -5.44    3.0
 11   -7.251338798703      -10.78       -5.93    3.0
 12   -7.251338798704      -11.82       -6.38    3.0
 13   -7.251338798705      -12.74       -7.31    2.0
 14   -7.251338798705   +  -14.57       -7.70    3.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.0683021318589466
[ Info: Arnoldi iteration step 2: normres = 0.5601654809290398
[ Info: Arnoldi iteration step 3: normres = 0.8260683837239154
[ Info: Arnoldi iteration step 4: normres = 0.27471260957331767
[ Info: Arnoldi iteration step 5: normres = 0.5406365348106889
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (3.06e-02, 7.71e-02, 4.52e-01, 2.85e-01, 1.68e-02)
[ Info: Arnoldi iteration step 6: normres = 0.2767706025903507
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (7.33e-03, 1.70e-01, 1.68e-01, 1.01e-01, 6.50e-02)
[ Info: Arnoldi iteration step 7: normres = 0.059304641607390574
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (2.06e-04, 9.12e-03, 7.13e-03, 2.51e-02, 4.76e-02)
[ Info: Arnoldi iteration step 8: normres = 0.10273273881326414
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (9.09e-